In [1]:
#import dependencies and assign API key
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint

eia_key = "3573bd97083abd7efae3399626d0ae59"

In [34]:
#identify series names and series ID for API data series call

#category API call
category_url = f"http://api.eia.gov/category/?api_key={eia_key}&category_id=404254"

#requests to get data from category url
data=requests.get(category_url).json()

#create empty lists for series_id and series_name
series_id=[]
series_name=[]

#iterate over the category and child series to extract 
for x in range(0, len(data['category']['childseries'])-1):
    series_id.append(data['category']['childseries'][x]['series_id'])
    series_name.append(data['category']['childseries'][x]['name'])

In [224]:
#place series ID and series names into a Dataframe
series_df = pd.DataFrame()
series_df["series_id"] = series_id
series_df["state summary"] = series_name
series_df.head()

,series_id,state summary
0,PET.C100000001.A,U.S. Total Gasoline All Sales/Deliveries by Pr...
1,PET.C100000001.M,U.S. Total Gasoline All Sales/Deliveries by Pr...
2,PET.C100010001.A,East Coast (PADD 1) Total Gasoline All Sales/D...
3,PET.C100010001.M,East Coast (PADD 1) Total Gasoline All Sales/D...
4,PET.C100011001.A,New England (PADD 1A) Total Gasoline All Sales...


In [226]:
#remove annual data from series_id's
remove_annual_df = series_df[~series_df["series_id"].str.contains(".A")]
remove_padd_df = remove_annual_df[~remove_annual_df["state summary"].str.contains("PADD")]
remove_padd_df = remove_padd_df.reset_index(drop=True)
remove_padd_df["state"] = remove_padd_df["state summary"].str.split('Total').str[0]
remove_padd_df.tail()

,series_id,state summary,state
46,PET.C100050041.M,Arizona Total Gasoline All Sales/Deliveries by...,Arizona
47,PET.C100050061.M,California Total Gasoline All Sales/Deliveries...,California
48,PET.C100050151.M,Hawaii Total Gasoline All Sales/Deliveries by ...,Hawaii
49,PET.C100050321.M,Nevada Total Gasoline All Sales/Deliveries by ...,Nevada
50,PET.C100050411.M,Oregon Total Gasoline All Sales/Deliveries by ...,Oregon


In [227]:
iter_ids = remove_padd_df["series_id"].values

In [236]:
new_df = pd.DataFrame()

data_url = f"http://api.eia.gov/series/?api_key={eia_key}&series_id=PET.C100000001.M"
series_data = requests.get(data_url).json()
st_cons = series_data['series'][0]['data']
new_df = pd.DataFrame(np.array(st_cons))
state_name = series_data['series'][0]['name']
new_df = new_df.rename(columns={0:"Month"})
new_df = new_df.rename(columns={1:state_name})
new_df.head()

,Month,"U.S. Total Gasoline All Sales/Deliveries by Prime Supplier, Monthly"
0,201812,363814.8
1,201811,372857.2
2,201810,378336.5
3,201809,371360.6
4,201808,390031.8


In [237]:
for id in iter_ids:
    
    data_url = f"http://api.eia.gov/series/?api_key={eia_key}&series_id={id}"
    series_data = requests.get(data_url).json()
    st_cons = series_data['series'][0]['data']
    df = pd.DataFrame(np.array(st_cons))
    state_name = series_data['series'][0]['name'].split("Total")
    df = df.rename(columns={0:"Month"})
    df = df.rename(columns={1:state_name[0]})
    new_df[f"{state_name[0]}"] = df[f"{state_name[0]}"]

In [242]:
new_df = new_df.drop(columns = ["U.S. Total Gasoline All Sales/Deliveries by Prime Supplier, Monthly"])
new_df

,Month,U.S.,Connecticut,Maine,Massachusetts,New Hampshire,Rhode Island,Vermont,Delaware,District of Columbia,...,Idaho,Montana,Utah,Wyoming,Alaska,Arizona,California,Hawaii,Nevada,Oregon
0,201812,363814.8,4226.8,1877.8,6387,1553.5,1753.4,809.7,1238.8,183.7,...,1944.4,1732.7,3512.2,868,633.2,7841.5,38591.6,1266.1,3131.8,4088.1
1,201811,372857.2,4227.6,1904.4,6459.8,1535.7,1735,807.4,1302.6,193.5,...,1923.3,1992.8,3664.1,870.5,625.6,8015.2,38553.6,1249.7,3141.5,4389.7
2,201810,378336.5,4165.6,2075.5,6578.2,1612.4,1810.8,895.8,1368,201.8,...,2066.1,2032.1,3798.2,1042.6,666.4,7996.7,39537.3,1275.6,3220.2,4464.4
3,201809,371360.6,4151.3,2034.3,6475.5,1538.2,1787,842,1334.7,205.9,...,2101.3,2166.4,3803,1088.5,700,8055.8,39825.4,1255.4,3260.5,4231.1
4,201808,390031.8,4359.6,2266.8,6797.6,1656.1,1950.6,925.4,1323.8,202.9,...,2164,2496.3,3895,1175.3,800.9,8344.7,41070.5,1297.7,3450.3,4504.4
5,201807,383738.2,4355.7,2193.5,6674.5,1625,1963,937.2,1363.1,209.8,...,2368.8,2577.5,3860.9,1215.3,802.7,7978.5,39835.3,1302.2,3340.3,4682.7
6,201806,391639.9,4479.9,2164.7,6793.7,1626.5,2166.2,881.4,1346.6,216,...,2273.4,2364.1,3783.1,1205.1,763.6,7973.7,40475.2,1291.3,3318.9,4558.5
7,201805,386016,4449.4,2024.7,6694.4,1462.2,1984.2,849.5,1498.5,214.2,...,2112.3,2153.1,3657.2,1103.2,721.1,8414.6,40136.2,1289.5,3336,4090
8,201804,374734.3,4196.5,1796.7,6361.5,1419.1,1887.6,744.7,1309.1,209.2,...,1996.6,1876.2,3529.1,963.3,627.1,8701.1,39905.6,1282.7,3306.5,4196.7
9,201803,374065.7,4171,1821.6,6194.4,1420.1,1796.1,802.5,1372,200.5,...,1965.5,1794.2,3731.4,972.1,655.4,8776.6,39429.8,1252.1,3195.1,3998.3
